# Scene Classification of Satellite Imagery using a Convolutional Neural Network (CNN) - A Tutorial

## Objective

The goal of this tutorial is to show you how a convolutional neural network (CNN) works by walking you through the steps involved in building, train, and deploy one. In this example, we will try to train a CNN so that it learns how to categorize a given image into one of 'Forest', 'River', 'Highway', 'AnnualCrop', 'SeaLake', 'HerbaceousVegetation', 'Industrial', 'Residential', 'PermanentCrop', 'Pasture'. In machine learning terminology, these are often called `classes`. So, given a satellite image, the CNN should output one of these categories or _classes_. This type of framework is called **_scene classification_** and in remote sensing, this is an important task often employed for agriculture monitoring, disaster management, land cover change, etc.

Let us start by importing some packages and libraries.

## Import necessary packages and libraries

In [ ]:
# future is imported to allow the use of different versions of Python
from __future__ import division, print_function, absolute_import
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) # ignore pesky warnings for this tutorial :)
warnings.filterwarnings("ignore", category=RuntimeWarning)

import os
import glob
import datetime
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
import xarray as xr

# Keras and tensorflow are libraries for deep learning which we will use to build the neural network
import keras
import tensorflow as tf
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten

In [ ]:
parent_dir = os.getcwd() # current directory, feel free to change it to whatever desired location

## The data

The data set comes courtesy of [EuroSAT](https://github.com/phelber/eurosat), put together using data from the [European Copernicus Sentinel-2 mission](https://dataspace.copernicus.eu/explore-data/data-collections/sentinel-data/sentinel-2) that utilizes wide-swath, high-resolution, multi-spectral imaging _"to support operational applications primarily for land services, including the monitoring of vegetation, soil and water cover, as well as the observation of inland waterways and coastal areas"_. 

![Sentinel satellite](assets/sentinel_satellite_with_logo.png)

## Read in the data

Before we start building a machine learning model, we must first always understand what the data looks like and what inherent properties we must be aware of. This, in turn, will help to determine the type of model and its structure. 

This particular data set we will see for this tutorial covers 13 spectral bands (we will only use the RGB channels) with 10 labeled classes on 27,000 geo-referenced images. Each image is 64 x 64 x 3 pixels (flattened). Let's load the netCDF4 file using `xarray`.

In [ ]:
sat_ds = xr.open_dataset('data/eurosat.nc')

## Visualize the data

Data visualization should almost always be the first step in any machine learning project. This is also sometimes referred to as **"Exploratory Data Analysis" or EDA**. Playing around with your data and visualizing it as images, histograms, plots, etc., will allow you to get a sense of what kinds of properties your data has (is it imbalanced? is it biased? is it corrupted? etc.). Once you know that, you can start cleaning up the data, doing any processing if needed before you move on to the algorithm stage.

### Imagery

Here, let's create a figure of our imagery in a 4 x 5 grid and sample randomly from our data set to display them as images.

In [ ]:
fig = plt.figure(figsize=(16, 16))
columns = 4
rows = 5
random_idx = np.random.randint(0, sat_ds.dims['n_imgs'] - 1, size=rows * columns) # random indices to select from sat_ds

for i in range(0, columns*rows):
    
    sample_ds = sat_ds.isel(n_imgs=random_idx[i])
    img = sample_ds['pixel_data'].values
    img = img.reshape((64, 64, 3)).astype('int') # because we had flattened it for xarray
    label = sample_ds['label'].values
    
    ax = fig.add_subplot(rows, columns, i + 1)
    ax.imshow(img, origin='lower')
    ax.set_title('Label = {}'.format(label))
    ax.set_xticks([])
    ax.set_yticks([])

plt.show()
plt.close()

### Statistics

Does our data set have the same number of images for every "class" or "label" or "scene"? Or is it imbalanced? Let's use a bar plot!

In [ ]:
# how many unique labels are there
labels, counts = np.unique(sat_ds['label'].values, return_counts=True)
n_labels = labels.size

# create some nice colors from matplotlib tableau colors
bar_colors = list(matplotlib.colors.TABLEAU_COLORS.values())

fig = plt.figure(figsize=(18, 6))
ax = fig.add_subplot(1, 1, 1)
ax.grid(axis='y', alpha=0.5) # set grid for just the y-axis with a 0.5 transparency
ax.bar(labels, counts, color=bar_colors, zorder=3) # plot it!
ax.set_ylabel('Number of samples available')
# no need to set x label as it should be self-evident
ax.set_title('Distribution of classes in the EuroSAT data set', fontweight='bold', fontsize=18) # make it bigger and bolder

plt.show()
plt.close()

We can see that most of the data set is more or less equally distributed across different classes. The `pasture` class has the lowest number of images to its name, all others have between 2,500 to 3,000 images.

_**Future work**_: Make a more comprehensive set of analyses on this data set. Here, we only went through a couple of analyses but you will want to be more thorough and detailed, especially if the data set is more complex than this one

## Reflect

Now that we have a visual and statistical sense of what our data set looks like, let's move on to other tasks. **Remember that we want our machine learning model (in this case, a CNN) to be able to look at one of these images and predict the _"label"_ or _"class"_.**

You may ask yourself - why is this important? Why do we want to build a machine learning model to tell us if the satellite is seeing a forest or a river if we already know what the answer? Doesn't this data set already have the labels?

Well, consider this - the data set here was manually labeled (which is often the case in building machine learning data sets) where researchers arduously went through tens of thousands of images and labeled them. It takes time and effort (not to mention the financial cost!) to build these huge data sets that ML models often require. And when you take into account that <ins>satellites like Sentintel-2 collect over 2.5 Tb of data every single day</ins>, we cannot possibly label every single image! So, if our ML model can learn on this data set, we can then apply it to new imagery to tell us what the satellite is seeing without having to spend years manually labeling our data.



## Prepare the data

### 1. Convert `labels` into numbered `classes`

We currrently have our `sat_ds` Xarray Dataset with pixel values in `pixel_data` and the labels (also called ground truth in ML) stored as strings like 'Forest', 'River', etc. But the model will expect a digital value, so to speak. So, let's map each of these labels to an integer value. The order doesn't matter since we will shuffle them anyway. These integer values are the _"answers"_ to each image that the model will predict eventually. They are called `classes`

In [ ]:
# create a dictionary mapping names to integers (or "classes")
label_to_class_map = {'Forest': 0,
                     'River': 1,
                     'Highway': 2,
                     'AnnualCrop': 3,
                     'SeaLake': 4,
                     'HerbaceousVegetation': 5,
                     'Industrial': 6,
                     'Residential': 7,
                     'PermanentCrop': 8,
                     'Pasture': 9
                    }

class_to_label_map ={0: 'Forest',
                     1: 'River',
                     2: 'Highway',
                     3: 'AnnualCrop',
                     4: 'SeaLake',
                     5: 'HerbaceousVegetation',
                     6: 'Industrial',
                     7: 'Residential',
                     8: 'PermanentCrop',
                     9: 'Pasture'}


def convert_scene_label_to_class_xarray(xr_ds):
    """ 
    Convert labels like Forest, River, etc. to integer labels (0, 1, ...) as classes
    and add it to the xarray dataset.
    """
    
    string_labels = xr_ds['label'].values
    
    # use np.vectorize to apply the mapping to the array
    int_array = np.vectorize(label_to_class_map.get)(string_labels)

    # add it to the xarray dataset
    xr_ds['class'] = xr.DataArray(int_array, dims=('n_imgs', ), coords=dict(n_imgs=('n_imgs', xr_ds.coords['n_imgs'].values)))

    return xr_ds

## Prepare the data

### 2. Split the data set

We need to split our data set into three different sets:

* training data set - this is the "labeled" data that the model will use to learn. it will look at each image in the training set, forward calculate the prediction, and then check that prediction against the given label using a loss function (also called error function or ~ cost function).
* validation data set - this is also "labeled" data but used for evaluating how well the model is learning. So, the model won't learn from the validation data, but by evaluating on it, you (the user) can get a sense of how well your model is built and start to debug if it isn't.
* testing data set - this is the data set which the model has never seen before and therefore provides a true measure of how well the model has been trained.

So, we need to split our large data set into these 3 components. Generally speaking, we want the training data set to be the largest, the testing data set to be the second largest, followed by the validation. But in ML, this is highly dependent on the actual data and the application and so on. For our use case on the EuroSAT data set, let's use a commonly followed split ratio of 80:10:10 meaning 80% of the data is for training, and 10% each for validation and testing. And let us also shuffle our data to eliminate any ordered biases.

In [ ]:
def train_val_test_split(xr_ds, val_fraction=0.1, test_fraction=0.1, normalize=True):
    """Splits the xarray dataset into training, validation, and testing sets.
    Note: Assumes that X = 'pixel_data', y = 'label'

    Args:
    ----
        xr_ds: xarray dataset

    Returns:
    -------
        X_train, X_val, X_test, y_train, y_val, y_test (ndarray): Split data into train, test and val (X and y).
    """

    # first, shuffle the data by simply creating indices and shuffling them 
    # let us also set a random seed to reproduce the shuffle every time
    np.random.seed(73) # in place

    n_imgs = xr_ds.sizes['n_imgs']
    ds_idx = np.arange(0, n_imgs, 1) # data set indices
    np.random.shuffle(ds_idx) # shuffle indices in-place

    X = xr_ds['pixel_data'].values
    y = xr_ds['class'].values

    # calculate the split index 
    split_index = int(n_imgs * (1 - test_fraction - val_fraction))
    n_train = split_index
    n_val   = int(n_imgs * val_fraction)
    n_test  = int(n_imgs * test_fraction)

    # split the data 
    X_train, y_train = X[ds_idx[0:split_index]], y[ds_idx[0:split_index]]

    X_val, y_val     = X[ds_idx[split_index:split_index+int(val_fraction * n_imgs)]], y[ds_idx[split_index:split_index+int(val_fraction * n_imgs)]]

    X_test, y_test   = X[ds_idx[split_index+int(val_fraction * n_imgs):split_index+int(val_fraction * n_imgs)+int(test_fraction * n_imgs)]], y[ds_idx[split_index+int(val_fraction * n_imgs):split_index+int(val_fraction * n_imgs)+int(test_fraction * n_imgs)]]

    # since our pixel data in the xarray dataset is all 1D, we need to shape them into 3D arrays of n_imgs x 64 x 64 x 3
    train_shape, val_shape, test_shape = (n_train, 64, 64, 3), (n_val, 64, 64, 3), (n_test, 64, 64, 3)
    X_train, X_val, X_test = X_train.reshape(train_shape), X_val.reshape(val_shape), X_test.reshape(test_shape)

    if normalize: # normalize by 255 i.e., the max value
        X_train, X_val, X_test = X_train/255, X_val/255, X_test/255

    return X_train, X_val, X_test, y_train, y_val, y_test

## Prepare the data

### 3. Make our `classes` probabilistic

This is the final data processing step. Since we have 10 different types of scenes, let us have our model generate a PDF spread across the classes. To do so, we must one-hot encode the `classes` which means to convert the 10 classes each to binary maps. For instance, if for a given image, the ground truth class is `4` (which is a `SeaLake` label), we need to assign a probability of 1 to that class, and a 0 to all the other classes. So, our `classes` which is of shape (`n_imgs`) will instead become (`n_imgs`, 10) meaning each image will have 10 probabilities whose sum will be 1.

Fortunately, `keras` offers a funtion `to_categorical` to do this easily.

In [ ]:
def one_hot_encode_classes(classes, num_classes):
    """
    One-hot encode classes into num_classes of binary probabilities
    """
    return keras.utils.to_categorical(classes, num_classes)

In [ ]:
# put it all together
validation_data_fraction = 0.1 # 10% of the data
test_data_fraction       = 0.1 # 10% of the data
num_classes = 10
normalize   = True # whether to normalize the pixels between 0 and 1


# Step 1: convert labels to classes
sat_ds = convert_scene_label_to_class_xarray(sat_ds)

# Step 2: split the data set
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(sat_ds, 
                                                                      val_fraction=validation_data_fraction, 
                                                                      test_fraction=test_data_fraction,
                                                                      normalize=normalize)

# Step 3: make the y classes probabilistic using one-hot encoding
y_train = one_hot_encode_classes(y_train, num_classes)
y_val   = one_hot_encode_classes(y_val, num_classes)
y_test  = one_hot_encode_classes(y_test, num_classes)

In [ ]:
# set a directory where the model will be saved
# if it doesn't exist, ensure that it will be created

model_dir = os.path.join(parent_dir, 'saved_models')
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

## Model - Convolutional Neural Network

The exciting part! Let's build a convolutional neural network (CNN)! Remember:

**Input**  = RGB images

**Output** = A set of probabilities (PDF) spread across 10 classes 


![CNN Architecture](assets/cnn_architecture.png)

### Convolution (Conv2D)

This layer creates a convolution kernel that is _convolved_ with whatever input is given to it from the previous stage over a 2D spatial (or temporal) dimension (height and width) to produce an output. It comes with an activation function which essentially applies a non-linear function to the neuron output. You could also make it linear but it is recommended to use non-linear activation functions like `tanh` (hyperbolic tangent) or `relu` (REctified Linear Unit).

### Max Pooling (MaxPooling2D)

Downsamples the input along its spatial dimensions (height and width) by only taking the maximum value over an input window (of size defined by `pool_size`) for each channel of the input. Downsampling helps CNNs learn the signal in the data better by filtering out the noise. 

### Dense (Fully connected layer)

The Dense layer (also called fully connected layer) implements the operation: `output = activation(dot(input, kernel) + bias)`. They connect every single input to the output and force the model to learn from every single one of them. The non-linear activation function then decides which ones are useful and which ones aren't. Dense layers capture and learn from the abstract feature representation by using a compressed version of the data.

In [ ]:
# Create the convolutional neural network that looks something like this:
# input -> convolution -> downsample -> convolution -> downsample -> convolution -> downsample -> dense -> dense

def model_architecture(input_data_shape, num_filters, kernel_size, num_dense_units, num_classes):
    # create a sequential model where the layers can be added easily.
    # for this example, we will use 3 convolutional layers, 2 pooling (downsampling) layers,
    # and 2 dense (fully connected) layers.
    # feel free to add or remove some layers to see what happens!
    
    model = keras.Sequential(
    [
        Input(shape=input_data_shape),
        Conv2D(num_filters[0], kernel_size=kernel_size, activation="relu"), # convolve
        MaxPooling2D(pool_size=(2, 2)), # halve the number of spatial dimensions
        Conv2D(num_filters[1], kernel_size=kernel_size, activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(num_filters[2], kernel_size=kernel_size, activation="relu"),
        MaxPooling2D(pool_size=(2, 2)), 
        Flatten(), # flatten due to keras backend requirements, not a learnable layer
        Dense(num_dense_units[0], activation="relu"), # the fully connected layer is called Dense in keras, 
        Dense(num_dense_units[1], activation="relu"), 
        Dense(num_classes, activation="softmax"), # we want probability as the output with num_classes
    ])

    return model

_**Future Work**_: Play around with the model by adding/removing layers. Change the kernel size, number of filters, padding, etc., to see how it influences the training and prediction performance.

# Hyperparameters
These are the "knobs" you can turn and change to your liking to optimize the training process. 

## Batch size

The number of images to be taken and trained before updating the weights. This is done to reduce the memory consumption. Instead of training all images at the same time, we do it batch-by-batch.

- Batch sizes are usually in powers of 2 e.g., 8, 16, 32, 64, 128 ...
- For example if your total number of training images = 2000 and batch size is 128 then we get 15 full batches and the final batch will have the remaining images.
- Higher batch size almost always gives better accuracies but will be computationally slow and take up more memory on your computer which can crash

## Learning rate 

Remember that CNNs learn by changing or updating the weights through backpropagating the error. So, if our weights are $w$, and the error is calculated as some function of our weights, $E(w)$, then using the error, the weights in the next time step are updated as:

$w_{t+1} = w_{t} - \alpha \dfrac{\partial{E}}{\partial{w}}$

The $\alpha$ is the learning rate. It is a number between 0 and 1 and dictates how fast your optimization moves. A popular optimization algorithm is gradient descent. Learning rates are usually set in tenths like 0.1, 0.01 etc,. It is a good idea to start with a low value and gradually increase or decrease.

## Number of epochs

1 epoch = the model has seen one full pass of the entire training set.
Since we use batches, 1 epoch will be competed after (dataset_size/batch_size) steps
Generally speaking, it is common to train for 100,000 or even 500,000 epochs. For this example we choose a small number.

## Tuning the hyperparameters
Feel free to play around with some of these "hyperparameters". For instance, you could modify the batch_size to be 8 instead of 16.

You could also run the entire model as is first and then reset, come back here and change something, run the model again to see what changes. For example,

* If the model is too slow to train, try decreasing the `batch_size`.
* If the loss is still high after training, try increasing the number of `epochs`
* If the model is learning too slowly or is not converging faster, try increasing the `learning_rate` (small steps)

    

In [ ]:
# define architecture details

input_shape = (64, 64, 3)    # because our images are 64 x 64 x 3 pixels across

# hyperparameters
num_filters = [16, 32, 64]   # filters for convolution it's usually a good idea to double the filters but feel free to play around with this
kernel_size = (3, 3)         # this is the convolution kernel
num_epochs  = 15             # number of epochs to train for
learning_rate = 0.001        # learning rate for the model weights. recommended to start with a low number like 0.0001
batch_size = 16              # number of images that the network will see at once. higher batch size = more memory required 
num_dense_units = [400, 200] # number of dense units for the dense layers


## Loss Function

The loss function (sometimes referred to as error or cost function) is one of the most important parts of a machine learning model. It dictates the metric by which your model will learn. For instance, if you are simply looking for the model to output a number (regression), you will want to use something like mean squared error (MSE). In our case, since we want to output probabilities, we need something different. That something could be a loss function called categorical cross entropy which looks like this:

If the probability that a given image belongs to a class $x$ is $p(x)$ and the CNN outputs a probability density function denoted as $\hat{p}(x)$, then we can write our cross entropy loss function simply as:

$Loss_{CE} = -\sum_{x\in classes} p(x) * log (\hat{p}(x))$

We will use the `model.compile` function in Keras to do this. 

## Train the CNN!

Right then, we've gone through all the data processing and model building stages. Let's put it all together and `train` our model. In other words, let's have the model learn on the (`X_train`, `y_train`) data set by _"fitting"_ the model to the data. This is the `model.fit` function built in to Keras.

We will also save the model using timestamps so that you can re-run this cell to re-train the model without having to rename it every time. And while we're at it, let's stop the model if it doesn't learn or improve for more than 15 epochs because that likely means that the model is stuck in a local minimum but thinks it is at the global minimum.

In [ ]:
# save the model with a date and time so you can run new models without having to change this name every time
model_name  = 'my_eurosat_cnn_{}.h5'.format(datetime.datetime.now().strftime('%Y-%m-%dT%H-%M-%S'))    # cnn will be saved under this filename

# build the model
model = model_architecture(input_shape, num_filters, kernel_size, num_dense_units, num_classes)

# callbacks to the model
# 1. stop the model early if it is not learning anything
# 2. save the model (checkpoint) only if the loss has improved i.e., if the model was able to learn something new or better
callbacks = [keras.callbacks.EarlyStopping(patience=15, verbose=1),
             keras.callbacks.ModelCheckpoint(filepath=os.path.join(model_dir, model_name), monitor="val_loss", save_best_only=True, verbose=1)]

print('Model will be saved to {}\n'.format(os.path.join(model_dir, model_name)))

# our cost function or loss function is cross-entropy which is probabalistic
# use the adaptive momentum optimizer algorithm
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# # fit the model to the dataset X: train_images, y: train_labels
history = model.fit(X_train, 
                    y_train, 
                    batch_size=batch_size, 
                    epochs=num_epochs, 
                    validation_data=(X_val, y_val),
                    verbose=1, 
                    callbacks=callbacks)

## Analyze how the training went

The training and validation loss should be close to each other and decrease over time (epochs).

* If training loss is low, but validation loss is much higher, that means the model _"overfitted"_
    * That means the model probably memorized the training data and when confronted with a new, unseen image, did not know how to respond

* If both losses are high, that means the model did not learn anything and _"underfitted"_

* If training loss is low, and validation loss is also low, that means the model did a good job

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 6))

# first plot will be of accuracies
ax[0].plot(history.history['accuracy'], color='teal', label='training')
ax[0].plot(history.history['val_accuracy'], color='orange', label='validation')
ax[0].set_title('Model Accuracy')
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('Epoch')
ax[0].legend()

# second plot will be of the errors
ax[1].plot(history.history['loss'], color='teal', label='training')
ax[1].plot(history.history['val_loss'], color='orange', label='validation')
ax[1].set_title('Model Loss')
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('Epoch')
ax[1].legend(['training', 'validation'])

plt.show()
plt.close()

## Test the trained model on test set

Now that the model has been trained, you can test it on new images i.e your test images. This is really what matters because it is a true test of how well the model can generalize to a never before seen set of images.

In [ ]:
val_score = model.evaluate(X_val, y_val, verbose=0)
test_score = model.evaluate(X_test, y_test, verbose=0)

print("Validation data set loss: {:0.2f}".format(val_score[0]))
print("Validation data set accuracy: {:0.2f}".format(val_score[1]))

print("\nTesting data set loss: {:0.2f}".format(test_score[0]))
print("Testing data set accuracy: {:0.2f}".format(test_score[1]))

In [ ]:
# use the model to predict on the testing data set
predictions = model.predict(X_test, verbose=0)

## Visualize the results

Let's also make it so that we draw randomly from our testing data set so that you can run the cell below repeatedly to produce different images and see how the model predicted on each of them

In [ ]:
# each time this cell is run, a different set of images will be displayed randomly

# display the results
fig = plt.figure(figsize=(16, 16))
columns = 4
rows = 4

random_samples = np.random.randint(0, len(X_test), size=rows * columns) # random indices to select from X_test

for i in range(0, columns*rows):
    ax = fig.add_subplot(rows, columns, i + 1)

    ax.imshow(X_test[random_samples[i]], origin='lower')
    predicted_label = class_to_label_map[np.argmax(predictions[random_samples[i]])]
    actual_label    = class_to_label_map[np.argmax(y_test[random_samples[i]])]
    confidence      = np.max(predictions[random_samples[i]]) # just the maximum probability
    ax.set_title('Predicted label = {}\nConfidence = {:0.2f}\nActual label = {}'.format(predicted_label, confidence, actual_label), pad=5, fontsize=12)
    ax.set_xticks([])
    ax.set_yticks([])
    
fig.subplots_adjust(hspace=0.5)
plt.show()
plt.close()

## Test it on random satellite images

Let's see if the model can identify what the scene is if it is given a random satellite image. There are a few such images in the `random_images/` directory. Let's read and make the model predict on them. The ground truth (defined vaguely here on purpose) is in the filename.

Note that this is an exercise to test the limits of the model and what data it was trained on. Not all satellite images are the same. Another consideration is the training data itself - the EuroSAT data had specific labeled scenes so anything outside of those regimes will likely confuse the model. Of course, maybe the model is so used to seeing data like that in the EuroSAT that it just memorized it and doesn't know what any other satellite images look like. Still, let's see if this breaks the model.

In [ ]:
# get filenames and therefore ground truths
rand_img_dir = 'random_images/'
random_imgs  = [f for f in os.listdir(rand_img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# create a 4 x 1 grid of images
fig = plt.figure(figsize=(16, 8))
columns = 4
rows = 1

for i in range(0, columns*rows):
    # first read in the image
    img = Image.open(os.path.join(rand_img_dir, random_imgs[i]))

    # if the image is not 64 x 64, then attempt to resize it
    if img.size != (64, 64):
        try:
            img = img.resize((64, 64))
        except Exception as err:
            print('Encountered the following error while resizing {}: {}'.format(os.path.join(rand_img_dir, random_imgs[i]), err))
            continue # skip this image then

    # for a single image, the model is expecting an input shape of (1, 64, 64, 3) so let's pad the first dimension
    img = np.array(img)
    img_for_model = np.expand_dims(img, axis=0)

    # and finally normalize it
    img_for_model = img_for_model/255

    # now let's use the model to predict on it
    img_prediction = model.predict(img_for_model, verbose=0)

    # same as before, convert numerical value to label and get the confidence i.e., max probability
    predicted_label = class_to_label_map[np.argmax(img_prediction)]
    actual_label    = random_imgs[i].split('.')[0] # just the filename itself without the extension
    confidence      = np.max(img_prediction)
    
    ax = fig.add_subplot(rows, columns, i + 1)
    ax.imshow(img, origin='lower')
    ax.set_title('Predicted label = {}\nConfidence = {:0.2f}\nActual label = {}'.format(predicted_label, confidence, actual_label), pad=5, fontsize=12)
    ax.set_xticks([])
    ax.set_yticks([])
    
# fig.subplots_adjust(hspace=0.5)
plt.show()
plt.close()

# Next Steps

Now that you've trained the first iteration of your model, how about we go check what happens when:
* you change the hyperparameters. Remember to change one thing at a time so you can keep track of which ones affect your model the most. For instance:
    * increase the `test_data_fraction` to 0.2 instead of 0.1
    * decrease the `batch_size` to 8
    * increase the `num_epochs` to 40
* add another or remove a convolutional layer to/from your model
* add another dense layer to your model
* normalize/un-normalize the input RGB images
* change the optimizer to "sgd" instead of "adam"

# Go Further

Here are some more examples to try out:

* A CNN to classify handwritten digits (you can upload a hand-drawn picture of a number to try out!): [MNIST Classifier](https://github.com/vikasnataraja/Handwritten-Digit-Classification-using-TensorFlow/blob/master/convolutional_network-mnist.ipynb)
* Several examples of deep learning in jupyter notebooks by Francois Challet, the creator of Keras: [Keras CNNs](https://github.com/fchollet/deep-learning-with-python-notebooks)

# References

Eurosat: A novel dataset and deep learning benchmark for land use and land cover classification. Patrick Helber, Benjamin Bischke, Andreas Dengel, Damian Borth. IEEE Journal of Selected Topics in Applied Earth Observations and Remote Sensing, 2019.

Introducing EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification. Patrick Helber, Benjamin Bischke, Andreas Dengel. 2018 IEEE International Geoscience and Remote Sensing Symposium, 2018.

Original Data Repo: https://github.com/phelber/eurosat